In this file I start by implementing a compressed sensing reconstruction approach. I want to have a well accepted classical method 
as a baseline for my later deep learning models.

In [ ]:
import sys
from pathlib import Path
import shutil
import numpy as np

In [2]:
import numpy as np
from pathlib import Path
import shutil

SUFFIX = "_B0corrected_wo_LipidMask"

def expand_mask_to_data(mask: np.ndarray, data_shape: tuple) -> np.ndarray:
    """Expandiert 3D-Masken auf 4D-Daten-Shape (broadcastbar)."""
    mask = np.asarray(mask, dtype=bool)
    if mask.ndim > len(data_shape):
        raise ValueError(f"Maske hat zu viele Dimensionen ({mask.ndim}) für Daten-Shape {data_shape}")
    if tuple(mask.shape) != tuple(data_shape[:mask.ndim]):
        raise ValueError(f"Masken-Shape {mask.shape} passt nicht zu den ersten Dimensionen der Daten {data_shape}")
    new_shape = mask.shape + (1,) * (len(data_shape) - mask.ndim)
    return mask.reshape(new_shape)

def combine_data(src_dir: Path, dst_dir: Path):
    src_masks = src_dir / "masks"
    src_data_dir = src_dir / "OriginalData"
    dst_masks = dst_dir / "masks"
    dst_data_dir = dst_dir / "OriginalData"

    # Zielstruktur anlegen
    dst_dir.mkdir(parents=True, exist_ok=True)
    dst_masks.mkdir(parents=True, exist_ok=True)
    dst_data_dir.mkdir(parents=True, exist_ok=True)

    # Maskenpfade
    brain_mask_path = src_masks / "brain_mask.npy"
    lipid_mask_path = src_masks / "lipid_mask.npy"

    # Datenpfade
    data_path = src_data_dir / "data.npy"
    b0_path = src_data_dir / "data_B0corrected.npy"

    # Laden
    data = np.load(data_path, allow_pickle=False)
    data_b0 = np.load(b0_path, allow_pickle=False)
    brain_mask = np.load(brain_mask_path, allow_pickle=False)
    lipid_mask = np.load(lipid_mask_path, allow_pickle=False)

    # Masken anpassen
    brain_mask_b = expand_mask_to_data(brain_mask, data.shape)
    lipid_mask_b = expand_mask_to_data(lipid_mask, data.shape)

    # Kombination
    combined = data_b0 * brain_mask_b + data * lipid_mask_b

    # Speichern
    np.save(dst_data_dir / "data.npy", combined)

    # Masken kopieren
    shutil.copy2(brain_mask_path, dst_masks / "brain_mask.npy")
    shutil.copy2(lipid_mask_path, dst_masks / "lipid_mask.npy")

# ==== HIER DEINE ORDNERLISTE FESTLEGEN ====
ordner_liste = [
    Path("Vol1_London"),
    Path("Vol2_London"),
    Path("Vol3_London"),
    Path("Vol4_London"),
    Path("Vol5_London"),
    Path("Vol5"),
    Path("Vol6"),
    Path("Vol7"),
    Path("Vol8"),
    Path("Vol9")
]

# Schleife ausführen
for src in ordner_liste:
    dst = src.parent / f"{src.name}{SUFFIX}"
    try:
        combine_data(src, dst)
        print(f"OK: {src} -> {dst}")
    except Exception as e:
        print(f"FEHLER bei {src}: {e}")


OK: Vol1_London -> Vol1_London_B0corrected_wo_LipidMask
OK: Vol2_London -> Vol2_London_B0corrected_wo_LipidMask
OK: Vol3_London -> Vol3_London_B0corrected_wo_LipidMask
OK: Vol4_London -> Vol4_London_B0corrected_wo_LipidMask
OK: Vol5_London -> Vol5_London_B0corrected_wo_LipidMask
OK: Vol5 -> Vol5_B0corrected_wo_LipidMask
OK: Vol6 -> Vol6_B0corrected_wo_LipidMask
OK: Vol7 -> Vol7_B0corrected_wo_LipidMask
OK: Vol8 -> Vol8_B0corrected_wo_LipidMask
OK: Vol9 -> Vol9_B0corrected_wo_LipidMask
